<hr style="border:3px solid coral"></hr>

# Cartesian communicators

<hr style="border:3px solid coral"></hr>


#### 1d case
When we distribute one dimensional data to multiple ranks, we divide up our Cartesian grid so that rank 0 is near the left edge of our domain, and the highest rank is at the right edge.  This is typically done using the rank to compute subintervals of the domain. 

    int N_local = N/nprocs;
    double dw = (b-a)/nprocs;
    double a_local = a + rank*dw;     # Processor rank used to determine subintervals of our 1d grid
    double b_local = a_local + dw;
    
Because we know how the data is laid out, we know how to communicate boundary condition data between processors.      
   
#### 2d case    
When we consider two dimensional domains, however, it is no longer so straightforward to tile the domain with processors.  Even among a few obvious choices, we would need to decide whether we 

* Tile in the x direction

* Tile in the y direction

* Use some other optimal "space filling" strategy for ensure optimal communication performance between ranks.

MPI has anticipated this problem, and provides routines to facilitate grid communication for higher dimensional grids.  These routines are 

* `MPI_Cart_create` : Create a virtual Cartesian communicator

* 'MPI_Cart_coords' : Obtain local coordinates within the communicator

* 'MPI_Cart_get' : Get information from Cartesian communicator

These routines allow us to set up a "virtual" network which allows us to communicate using an $(I,J)$ grid of virtual ranks.  Moreover, virtual communicators will allow us to easily handle periodic data. 

Read more about [Virtual topologies](https://hpc-tutorials.llnl.gov/mpi/virtual_topologies/).

<hr style="border:3px solid coral"></hr>

## What is a Cartesian communicator?

<hr style="border:3px solid coral"></hr>

So far, we have been using a default communicator `MPI_WORLD_COMM`.  This object stores information about the number of processors available,  the current rank, and is passed to most other MPI routines involved in data communication.  

The communicator can tell us

* How many ranks are currently being used (e.g. `nprocs')

* The current rank (e.g. `rank'`)





<hr style="border:2px solid coral"></hr>

## Shift data and replace in a 1d array

<hr style="border:2px solid coral"></hr>

In this example, each processor in a 1d array stores a single integer value.  We create a Cartesian communicator and then shift each value to a neighboring processor. 

We use the MPI routine `MPI_Cart_create`, with signature

    int MPI_Cart_create ( MPI_Comm comm_old, int ndims, int *dims, int *periods,
                                int reorder, MPI_Comm *comm_cart )
                                
where

* **comm_old** is the existing communicator (e.g. `MPI_COMM_WORLD`)

* **ndims** Grid dimensions (1,2,3,...)

* **dims** an array determine size of grid in each direction.  **dims** should be an array of dimension `ndims`. 

* **periods**  Periodicity of each direction. 

* **reorder** Reorder ranks (set to 0 or 1)

* **comm_cart** Address of new communicator. 

Once we create a new communicator, we can no longer rely on our own rank layout, but instead must query the communicator to get the rank of neighboring processors.  We will get this neighboring rank information using the `MPI_Cart_shift` routine.

    int MPI_Cart_shift ( MPI_Comm comm, int direction, int displ,
                               int *source, int *dest )
                               
* **comm** The newly created communicator

* **direction** The direction to shirt (+1, -1)

* **displ** Displacement value

* **source** Source rank (an output value)

* **dest** Destination rank (an output value)


In the example below, we  shift an integer value to the left or right in our original array. 

* A "displacement" value determines whether we shift right or left. 

    * If **displacement** is 1, each processor sends its value to its right neighbor

    * If **displacement** is -1, each processor sends its value to its left neighbor

If the domain is **periodic**, each processor has a neighbor.  If the domain is not periodic, then the left most processor does not have a left neighbor, and the rightmost processor does not have a right neighbor. 

We also use `MPI_Sendrecv_replace`, which combines a the send buffer and the receive buffer.  The send/receives are coordinated so that the send occurs before the value in the send buffer is replaced with a received value. 

In [1]:
%%file cart_01.c

#include <stdio.h>
#include <stdlib.h>

#include <mpi.h>


int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    // # Each processor stores a single integer
    int myval = rank + 10;
    
    int arr[nprocs];
    MPI_Gather(&myval,1,MPI_INTEGER,&arr[0],1,MPI_INTEGER,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("Before shift : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",p);
        printf("\n");
        
        printf("%8s : ","Value");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",arr[p]);
        printf("\n");
    }

    

    // #  Create a new communicator
    MPI_Comm comm_cart;  
    int ndim = 1;
    int dims[1] = {nprocs};
    int reorder = 0;
    
    
    int periodicity[1] = {0};
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    // # Shift information : 1: receive value from the left; -1
    int displ = 1;
    
    
    // #  source : This rank will receive data from SOURCE
    // #  dest   : This rank will send data to DESTINATION
    
    int dir = 0;
    int source, dest;
    MPI_Cart_shift(comm_cart, dir, displ, &source, &dest);
    
    int tag = 0;
    int myval_before = myval;
    // # Only a send buffer is required, no recv. buffer.
    MPI_Sendrecv_replace(&myval, 1, MPI_INTEGER, dest, tag, 
                                    source, tag, comm_cart, MPI_STATUS_IGNORE);

    if (rank == 0)
    {
        printf("\n");        
        
        if (displ== 1)
            printf("Values are shifted to the right (displ = 1). ",displ);
        else
            printf("Values are shifted to the left (displ = -1). ",displ);
        
        if (periodicity[0] == 1)
            printf("Domain is periodic\n\n");
        else
            printf("Domain is not periodic\n\n");
    }
    
    MPI_Gather(&myval,1,MPI_INTEGER,arr,1,MPI_INTEGER,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("After shift : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",p);
        printf("\n");
        
        printf("%8s : ","Value");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",arr[p]);
        printf("\n");
    }

    MPI_Finalize();
}

Overwriting cart_01.c


In [3]:
%%bash 

mpicc -o cart_01 cart_01.c

mpirun -n 8 cart_01

cart_01.c:71:68: warning: data argument not used by format string [-Wformat-extra-args]
            printf("Values are shifted to the right (displ = 1). ",displ);
                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^
cart_01.c:73:68: warning: data argument not used by format string [-Wformat-extra-args]
            printf("Values are shifted to the left (displ = -1). ",displ);
                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^
2 warnings generated.
[proxy:0:0@GRAD-PHDCOMP-02] HYDU_create_process (utils/launch/launch.c:73): execvp error on file cart_01 (No such file or directory)
[proxy:0:0@GRAD-PHDCOMP-02] HYDU_create_process (utils/launch/launch.c:73): execvp error on file cart_01 (No such file or directory)
[proxy:0:0@GRAD-PHDCOMP-02] HYDU_create_process (utils/launch/launch.c:73): execvp error on file cart_01 (No such file or directory)
[proxy:0:0@GRAD-PHDCOMP-02] HYDU_create_process (utils/launch/launch.c:73): execvp error on file cart_01 (No such f

CalledProcessError: Command 'b'\nmpicc -o cart_01 cart_01.c\n\nmpirun -n 8 cart_01\n'' returned non-zero exit status 255.

#### Question

What happens to values at either end of the array? 

<hr style="border: 2px solid coral"></hr>

# Filling ghost cell values (1d)

<hr style="border: 2px solid coral"></hr>

The Cartesian communicator is particularly useful in filling ghost cells in gridded data from neighboring ranks. 

Below, each processor creates a length 3 array of values $x_{j}$, $j = -1,0,1$.  Each processor computes a value at $x_0$.  

Viewing these values as values of the function $f(x) = x$, each processor computes a value $f(x_0)$.  We can then approximate a derivative $f'(x) = 1$ at $x_0$ using the centered formula

\begin{equation}
x_0 = \frac{x_{1} - x_{-1}}{2h}
\end{equation}

where $x_0$ is local to each rank.  To obtain ghost values at $x_{-1}$ and $x_1$ use `MPI_Sendrecv`.  

To get the source and destination ranks needed for `MPI_Sendrecv`, we create a Cartesian communicator, and use `MPI_Cart_shift` to determine source and destination ranks.

<center>
<table>
    <tr>
        <td><img width=300 style="padding-top:20px" src="./cart_01.png"></img></td>
        <td><img width=300 style="padding-top:20px" src="./cart_02.png"></img></td>
    </tr>
</table>    
</center>

To fill all ghost cells, we need to do both a left shift and a right shift. 


In [ ]:
# Leave blank

In [4]:
%%file cart_02.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    // # Fill in q[-1] and q[1] with data from neighbors */
    MPI_Comm comm_cart;
    int ndim = 1;
    int dims[1] = {nprocs};
    int periodicity[1] = {0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    
    double qmem[3];    
    double *q = &qmem[1];

    double h = 1.0/nprocs;  
    
    int mycoords[1];
    MPI_Cart_get(comm_cart,ndim,dims,periodicity,mycoords);
    
    // # Fill in cell centered mesh locations using "my_coords"
    q[0] = (mycoords[0]+0.5)*h;
    
    double arr[nprocs];
    MPI_Gather(q,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("f(x) = x (cell centers) : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }        

    // #  Fill ghost value q[-1]  (shift values right)
    int dir = 0;
    int disp = 1;
    int source, dest;
    int ierr = MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);
    
    //# Need to check if we have valid source terms
    if (source == MPI_PROC_NULL)
    {
        //#we are at a left boundary
        q[-1] = -h/2;
    }

    int tag = 0;
    MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                 &q[-1], 1, MPI_DOUBLE, source, tag, 
                 comm_cart, MPI_STATUS_IGNORE);

    // #  Fill ghost value q[1]
    disp = -1;  
    MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);
    
    if (source == MPI_PROC_NULL)
    {
        //#we are at a right boundary
        q[1] = 1+h/2;
    }


    tag = 1;
    ierr = MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                        &q[1], 1, MPI_DOUBLE, source, tag, 
                        comm_cart, MPI_STATUS_IGNORE);

    // # Compute a derivative using left and right values
    double deriv = (q[1] - q[-1])/(2*h);
    
    MPI_Gather(&deriv,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("\n");
        printf("Derivative f'(x) = 1 (cell-centers): \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f'(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }
    

    MPI_Finalize();
}

Overwriting cart_02.c


In [5]:
%%bash 

mpicc -o cart_02 cart_02.c

mpirun -n 8 cart_02
./cart_02

[mpiexec@GRAD-PHDCOMP-02] control_cb (pm/pmiserv/pmiserv_cb.c:206): assert (!closed) failed
[mpiexec@GRAD-PHDCOMP-02] HYDT_dmxu_poll_wait_for_event (tools/demux/demux_poll.c:76): callback returned error status
[mpiexec@GRAD-PHDCOMP-02] HYD_pmci_wait_for_completion (pm/pmiserv/pmiserv_pmci.c:160): error waiting for event
[mpiexec@GRAD-PHDCOMP-02] main (ui/mpich/mpiexec.c:325): process manager error waiting for completion


CalledProcessError: Command 'b'\nmpicc -o cart_02 cart_02.c\n\nmpirun -n 8 cart_02\n./cart_02\n'' returned non-zero exit status 255.

### Question

* How can we fix the derivatives at the endpoints? 

<hr style="border: 2px solid coral"></hr>

# Filling ghost cell values (2d)

<hr style="border: 2px solid coral"></hr>

Where the real power of the Cartesian communicator comes in is for higher dimensional grids.  

Below, we alllocate a 2d array, and use a two dimensional communicator. 

To facilitate using finite difference schemes in 2d, we use the same trick as in 1d.  

<center>
<table>
    <tr>
        <td><img width=400 style="padding-top:20px" src="./cart_03.png"></img></td>
        <td><img width=400 style="padding-top:20px" src="./cart_04.png"></img></td>
    </tr>
</table>    
</center>

This allows us to apply finite difference stencils in two dimensions. 

    

In [ ]:
%%file cart_03.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    
    // # Create a 3x3 block of memory
    double   *qmem = malloc(3*3*sizeof(double));
    double **qrows = malloc(3*sizeof(double*));

    for(int i = 0; i < 3; i++)
        qrows[i] = &qmem[3*i + 1];
    double  **q = &qrows[1];
    
    double qmem[3][3];    
    double *q = &qmem[1];

    double h = 1.0/nprocs;  
        
    double arr[nprocs];
    MPI_Gather(q,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("f(x) = x (cell centers) : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }        

    // # Fill in q[-1] and q[1] with data from neighbors */
    MPI_Comm comm_cart;
    int ndim = 1;
    int dims[1] = {nprocs};
    int periodicity[1] = {0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    // #  Fill ghost value q[-1]  (shift values right)
    int dir = 0;
    int disp = 1;
    int source, dest;
    int ierr = MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    int tag = 0;
    MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                 &q[-1], 1, MPI_DOUBLE, source, tag, 
                 comm_cart, MPI_STATUS_IGNORE);

    // #  Fill ghost value q[1]
    disp = -1;  
    MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    tag = 1;
    ierr = MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                        &q[1], 1, MPI_DOUBLE, source, tag, 
                        comm_cart, MPI_STATUS_IGNORE);

    // # Compute a derivative using left and right values
    double deriv = (q[1] - q[-1])/(2*h);
    
    MPI_Gather(&deriv,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("\n");
        printf("Derivative f'(x) = 1 (cell-centers): \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f'(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }
    

    MPI_Finalize();
}